In [10]:
import torch

count=1
torch.manual_seed(1)
z=torch.FloatTensor([1,2,3])
hypothesis=torch.nn.functional.softmax(z,dim=0)
print("{}. {}".format(count,hypothesis)) # 1. softmax 함수 : e^x_k/e^x_1+...+e^x_n
count+=1
print("{}. {}".format(count,hypothesis.sum())) # 2. softmax 함수의 합은 1
count+=1

z=torch.rand(3,5,requires_grad=True)
hypothesis=torch.nn.functional.softmax(z,dim=1)
print("{}. {} {}".format(count,hypothesis,hypothesis.shape)) # 3. 입력의 형태와 동일
count+=1
print("{}. {:.2f} [0]: {:.2f} [1]: {:.2f} [2]: {:.2f}".format(count,hypothesis.sum(),
hypothesis[0].sum(),hypothesis[1].sum(),hypothesis[2].sum())) # 4. 가장 하위 차원의 합 1
count+=1

y=torch.randint(5,(3,)).long()  # 입력데이터 결과값
print("{}. {} {}\n {} {}".format(count,y,y.shape,y.unsqueeze(1),y.unsqueeze(1).shape))  # 5.
count+=1

y_one_hot=torch.zeros_like(hypothesis)
y_one_hot.scatter_(1,y.unsqueeze(1),1)
print("{}. {}".format(count,y_one_hot)) # 6.
count+=1

cost=(y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
print("{}. {}".format(count,cost)) # 7.

1. tensor([0.0900, 0.2447, 0.6652])
2. 1.0
3. tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>) torch.Size([3, 5])
4. 3.00 [0]: 1.00 [1]: 1.00 [2]: 1.00
5. tensor([0, 2, 1]) torch.Size([3])
 tensor([[0],
        [2],
        [1]]) torch.Size([3, 1])
6. tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])
7. 1.468920350074768


In [9]:
import torch

count=1
torch.manual_seed(1)
z=torch.rand(3,5,requires_grad=True)
hypothesis=torch.nn.functional.softmax(z,dim=1)
y=torch.randint(5,(3,)).long() 
y_one_hot=torch.zeros_like(hypothesis)
y_one_hot.scatter_(1,y.unsqueeze(1),1)

cost=(y_one_hot*-torch.log(torch.nn.functional.softmax(z, dim=1))).sum(dim=1).mean()
print("{}. {}".format(count,cost)) # 1.
count+=1

cost=(y_one_hot*-torch.nn.functional.log_softmax(z, dim=1)).sum(dim=1).mean()
print("{}. {}".format(count,cost)) # 2.
count+=1

cost=torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(z, dim=1), y)
print("{}. {}".format(count,cost)) # 3.
count+=1

cost=torch.nn.functional.cross_entropy(z, y)
print("{}. {}".format(count,cost)) # 4.
count+=1



1. 1.468920350074768
2. 1.468920350074768
3. 1.468920350074768
4. 1.468920350074768


In [36]:
import torch

x_train=[[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_train=[2,2,2,1,1,1,0,0]
x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)

W=torch.zeros((4,3),requires_grad=True)
b=torch.zeros(1,requires_grad=True)
optimizer=torch.optim.SGD([W,b],lr=0.1)
nb_epochs=1000
ver=2
for epoch in range(nb_epochs+1):
    z=x_train.matmul(W)+b
    # cost ver_1
    if ver==1:
        hypothesis=torch.nn.functional.softmax(z,dim=1)
        y_one_hot=torch.zeros_like(hypothesis)
        y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
        cost=(y_one_hot*-torch.log(torch.nn.functional.softmax(hypothesis,dim=1))).sum(dim=1).mean()
    # cost ver_2
    elif ver==2:
        hypothesis=torch.nn.functional.softmax(z,dim=1)
        cost=torch.nn.functional.cross_entropy(z, y_train) # binary_cross_entropy 와 다르게 softmax를 하지 않고 넣어도 됨
    # cost ver_3
    elif ver==3:
        hypothesis=torch.nn.functional.softmax(z,dim=1)
        criterion=torch.nn.CrossEntropyLoss()
        cost=criterion(z,y_train) # binary_cross_entropy 와 다르게 softmax를 하지 않고 넣어도 됨
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%200==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch,nb_epochs,cost.item()
        ))

prediction=(hypothesis>=torch.FloatTensor([0.5])).int()
prediction=(prediction==1).nonzero(as_tuple=False)[:,1]
for i in range(len(prediction)):
    correct_prediction=prediction[i]==y_train[i]
    print("{}. correct: {} prediction: {} y_train: {}".format(
        i,correct_prediction,prediction[i],y_train[i]))

Epoch    0/1000 Cost: 1.098612
Epoch  200/1000 Cost: 0.839114
Epoch  400/1000 Cost: 0.788472
Epoch  600/1000 Cost: 0.764449
Epoch  800/1000 Cost: 0.749399
Epoch 1000/1000 Cost: 0.738749
0. correct: True prediction: 2 y_train: 2
1. correct: True prediction: 2 y_train: 2
2. correct: True prediction: 2 y_train: 2
3. correct: True prediction: 1 y_train: 1
4. correct: False prediction: 0 y_train: 1
5. correct: True prediction: 1 y_train: 1
6. correct: True prediction: 0 y_train: 0
7. correct: True prediction: 0 y_train: 0


## High-level Implementation with `nn.Module`

In [14]:
import torch

class SoftmaxClassifierModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=torch.nn.Linear(4,3)

    def forward(self,x):
        return self.linear(x)

x_train=[[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_train=[2,2,2,1,1,1,0,0]
x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)

W=torch.zeros((4,3),requires_grad=True)
b=torch.zeros(1,requires_grad=True)
model=SoftmaxClassifierModel()
optimizer=torch.optim.SGD(model.parameters(),lr=0.1)
nb_epochs=1000
for epoch in range(nb_epochs + 1):
    prediction=model(x_train)
    cost=torch.nn.functional.cross_entropy(prediction,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%200==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch,nb_epochs,cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358734
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
